In [21]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm, trange
from math import sqrt
import math
import pandas as pd
import numpy as np
import heapq
import random

In [22]:
n,m=943,1682
cols = ['user_id', 'item_id', 'rating', 'ts']
base = pd.read_csv('../../ml-100k/u1.base',sep='\t',names=cols)
test = pd.read_csv('../../ml-100k/u1.test',sep='\t',names=cols)

In [23]:
# 创建uesr-item矩阵
train_matrix = np.zeros((n, m))
test_matrix = np.zeros((n, m))
y_ui = np.zeros((n, m))
for i in base.itertuples():
    if i[3]>=4:
        train_matrix[i[1] - 1, i[2] - 1] = i[3]

for i in test.itertuples():
    if i[3]>=4:
        test_matrix[i[1] - 1, i[2] - 1] = i[3]

# 创建y矩阵
for i in range(n):
    for j in range(m):
        if train_matrix[i,j] >= 4:
            y_ui[i,j] = 1

In [24]:
# 相关变量初始化
idx = np.where(train_matrix!=0)
miu = np.mean(train_matrix[idx[0],idx[1]])
ROEM = np.zeros((n,m))
row_total_y_ui = sum(y_ui)

In [25]:
for i in range(n):
    ROEM[i] = row_total_y_ui/n-miu

In [26]:
Ire = list()
for i in tqdm(range(n)):
    exclude = list(np.where(train_matrix[i]!=0)[0])
    ls = list()
    for j in range(m):
        if j in exclude:
            continue
        ls.append((j,ROEM[i,j]))
    ls = sorted(ls,key= lambda x: x[1], reverse=True)
    Ire.append([ls[j][0] for j in range(len(ls))])

100%|████████████████████████████████████████████████████████████████████████████████| 943/943 [00:12<00:00, 75.00it/s]


In [7]:
def PreK(rank, truth, k):
    prek = 0
    ure = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        preuk = 0
        ure+=1
        for j in range(k):
            if rank[i][j] in ls:
                preuk+=1
        prek+=preuk/k
    return prek/ure

In [8]:
def RecK(rank, truth, k):
    reck = 0
    ure = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        recuk = 0
        ure += 1
        for j in range(k):
            if rank[i][j] in ls:
                recuk += 1
        reck+=recuk/ts
    return reck/ure

In [9]:
def F1K(rank, truth, k):
    f1k = 0
    ure = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        num = 0
        ure += 1
        for j in range(k):
            if rank[i][j] in ls:
                num += 1
        recuk = num/ts
        preuk = num/k
        if preuk+recuk != 0:
            f1k+=2*(preuk*recuk)/(preuk+recuk)
    return f1k/ure

In [62]:
def NDCGK(rank, truth, k):
    ndcgk = 0
    ure = 0
    max_DCGuK = [0]*6
    for i in range(1,k+1):
        max_DCGuK[i] = max_DCGuK[i-1]
        max_DCGuK[i] += 1/math.log(i+1)
    print(max_DCGuK)
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        DCGuK = 0
        ure+=1
        num = 0
        for j in range(k):
            if rank[i][j] in ls:
                DCGuK+=(np.power(2,1)-1)/math.log(j+2)
                num+=1
        if num==0:
            continue
        else:
            if ts>=5:
                ndcgk+=DCGuK/max_DCGuK[5]
            else:
                ndcgk+=DCGuK/max_DCGuK[num]
#                 print("u:"+str(i))
#                 print("max_DCGuK:"+str(max_DCGuK[num]))
    return ndcgk/ure

In [63]:
NDCGK(Ire,test_matrix,5)

[0, 1.4426950408889634, 2.352934267515801, 3.074281787960283, 3.695616722519895, 4.253727349071142]


0.2627271458384939

In [64]:
0.2627271458384939-0.2568

0.005927145838493908

In [56]:
def NDCGK(rank, truth, k):
    ndcgk = 0
    ure = 0
    max_DCGuK = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        DCGuK = 0
        ure+=1
        for j in range(k):
            if rank[i][j] in ls:
                DCGuK+=(np.power(2,1)-1)/np.log2(j+2)
        if max_DCGuK<DCGuK:
            max_DCGuK = DCGuK
        ndcgk+=DCGuK/max_DCGuK
    return ndcgk/ure

In [8]:
def CallK(rank, truth, k):
    callk = 0
    ure = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        calluk = 0
        ure+=1
        for j in range(k):
            if rank[i][j] in ls:
                calluk += 1
        break
        if calluk>0:
            callk+=1
    return callk/ure

In [7]:
def MRR(rank, truth):
    mrr = 0
    ure = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        ure+=1
        for j in range(len(rank[i])):
            if rank[i][j] in ls:
                mrr+=1/(j+1)
                break
    return mrr/ure

In [13]:
def MAP(rank, truth):
    map_ = 0
    ure = 0
    for u in range(n):
        ap = 0
        ls = list(np.where(truth[u]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        ure+=1
        num = len(rank[u])
        for i in range(ts):
            apu = 0
            rank_i = 0
            rank_j = 0
            if ls[i] in rank[u]:
                rank_i = rank[u].index(ls[i])+1
            for j in range(ts):
                if ls[j] in rank[u]:
                    rank_j = rank[u].index(ls[j])+1
                if (rank_i>rank_j) and (ls[i] in rank[u]) and (ls[j] in rank[u]):
                    apu+=1
            if ls[i] in rank[u]:
                apu += 1
                apu /= float(rank_i)
                ap += apu
        ap /= ts
        map_ += ap
    return map_/ure

In [9]:
def ARP(rank, truth):
    arp = 0
    ure = 0
    I = len(np.where(sum(train_matrix+test_matrix)!=0)[0])
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        ure+=1
        tm = list(set(rank[i]).intersection(set(ls)))
        rp = 0
        Iu = len(np.where(y_ui[i]!=0)[0])
        if len(tm)>=1:
            for item in tm:
                pui = rank[i].index(item)+1
                rp += float(pui)/(I-Iu)
            rp /= ts
            arp += rp
    return arp/ure

In [15]:
def AUC(rank, truth, R):
    auc = 0
    ure = 0
    for u in range(n):
        ls = list(np.where(truth[u]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        ure+=1
        aucu = 0
        len_rte = 0
        for i in ls:
            for j in rank[u]:
                if j not in ls:
                    if R[u][i]>R[u][j]:
                        aucu += 1
                    len_rte += 1
        if len_rte > 0:
            aucu /= len_rte
            auc += aucu
    return auc/ure

In [16]:
print("Pre@5:"+str(PreK(Ire,test_matrix,5)))
print("Rec@5:"+str(RecK(Ire,test_matrix,5)))
print("F1@5:"+str(F1K(Ire,test_matrix,5)))
print("NDCG@5:"+str(NDCGK(Ire,test_matrix,5)))# ???
print("1-call@5:"+str(CallK(Ire,test_matrix,5)))
print("MRR:"+str(MRR(Ire,test_matrix)))
print("MAP:"+str(MAP(Ire,test_matrix)))
print("ARP:"+str(ARP(Ire,test_matrix)))
print("AUC:"+str(AUC(Ire, test_matrix, ROEM)))# ???

Pre@5:0.23377192982456205
Rec@5:0.05712433087638161
F1@5:0.0774722624073191
NDCG@5:0.25702211962849764
1-call@5:0.5877192982456141
MRR:0.4656607532775578
MAP:0.15157217479577773
ARP:0.15489890941253937
AUC:0.8703130296701466


In [10]:
pd.DataFrame(Ire)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,99,173,257,97,285,312,55,317,299,287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49,180,173,257,97,171,312,55,78,317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49,99,126,173,257,0,97,285,171,312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49,180,99,126,173,0,97,285,171,312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99,126,257,0,97,285,312,55,78,317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,49,180,99,173,0,97,285,171,312,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,180,99,126,0,285,78,317,287,167,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,49,99,126,173,97,285,171,312,55,78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,180,99,126,0,97,285,312,55,172,287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
